<a href="https://colab.research.google.com/github/AngelRosalesContreras/Practica_1-3/blob/main/Practica_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica 1.3 (Transformada de Hough y Etiquetado de componentes conectados)**

In [ ]:
!pip install anytree

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from anytree import Node, RenderTree

In [ ]:
def find(parent, i):
    if parent[i] == -1:
        return i
    return find(parent, parent[i])

def union(parent, x, y):
    x_set = find(parent, x)
    y_set = find(parent, y)
    if x_set != y_set:
        parent[x_set] = y_set

def build_component_tree(imagen_binarizada):
    height, width = imagen_binarizada.shape
    parent = [-1] * (height * width)
    component_colors = {}

    component_count = 0

    for y in range(height):
        for x in range(width):
            if imagen_binarizada[y, x] == 255:
                neighbors = []
                if x > 0:
                    neighbors.append((y, x - 1))
                if y > 0:
                    neighbors.append((y - 1, x))

                if not neighbors:
                    component_count += 1
                    component_colors[component_count] = np.random.randint(0, 256, 3)
                    imagen_binarizada[y, x] = component_count

                else:
                    neighbor_labels = []
                    for neighbor_y, neighbor_x in neighbors:
                        neighbor_label = imagen_binarizada[neighbor_y, neighbor_x]
                        if neighbor_label != 0:
                            neighbor_labels.append(neighbor_label)

                    if not neighbor_labels:
                        component_count += 1
                        component_colors[component_count] = np.random.randint(0, 256, 3)
                        imagen_binarizada[y, x] = component_count
                    else:
                        min_label = min(neighbor_labels)
                        imagen_binarizada[y, x] = min_label
                        for label in neighbor_labels:
                            if label != min_label:
                                union(parent, label, min_label)

    # Crear nodos para los componentes
    component_nodes = {}
    root = Node("Root")
    for i in range(1, component_count + 1):
        parent_label = find(parent, i)
        component_nodes[i] = Node(f'Componente {i}: {np.sum(imagen_binarizada == i)} píxeles', parent=component_nodes.get(parent_label, root))

    return root

def print_component_tree(root):
    for pre, _, node in RenderTree(root):
        print(f"{pre}{node.name}")

def label_connected_components(image_path):
    imagen = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, imagen_binarizada1 = cv2.threshold(imagen, 20, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(imagen_binarizada1)

    tamano_maximo = 10000

    for label in range(1, num_labels):
        if stats[label, cv2.CC_STAT_AREA] > tamano_maximo:
            labels[labels == label] = 0

    imagen_binarizada = np.where(labels > 0, 255, 0).astype(np.uint8)

    component_tree = build_component_tree(imagen_binarizada)
    print_component_tree(component_tree)

image_path = 'placas/103.JPG'
label_connected_components(image_path)


Root
├── Componente 1: 915 píxeles
├── Componente 2: 443 píxeles
├── Componente 3: 393 píxeles
├── Componente 4: 49 píxeles
├── Componente 5: 11 píxeles
├── Componente 6: 938 píxeles
├── Componente 7: 55 píxeles
├── Componente 8: 13 píxeles
├── Componente 9: 64 píxeles
├── Componente 10: 68 píxeles
├── Componente 11: 61 píxeles
├── Componente 12: 33 píxeles
├── Componente 13: 11 píxeles
├── Componente 14: 69 píxeles
├── Componente 15: 42 píxeles
├── Componente 16: 43 píxeles
├── Componente 17: 17 píxeles
├── Componente 18: 22 píxeles
├── Componente 19: 6 píxeles
├── Componente 20: 456 píxeles
├── Componente 21: 12 píxeles
├── Componente 22: 4 píxeles
├── Componente 23: 32 píxeles
├── Componente 24: 264 píxeles
├── Componente 25: 18 píxeles
├── Componente 26: 29 píxeles
├── Componente 27: 12 píxeles
├── Componente 28: 19 píxeles
├── Componente 29: 28 píxeles
├── Componente 30: 149 píxeles
├── Componente 31: 27 píxeles
├── Componente 32: 13 píxeles
├── Componente 33: 63 píxeles
├── Compo

# Práctica 1.3 (Transformada de Hough y Etiquetado de componentes conectados)

Ponemos las librerias necesarias

In [ ]:
import cv2
import numpy as np
import random
from google.colab.patches import cv2_imshow

Desarrollamos Canny

In [ ]:
# Función para aplicar Canny
def canny_edge_detection(image, low_threshold, high_threshold):
    # Convertir la imagen a escala de grises
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Aplicar el filtro de Sobel para encontrar gradientes en la imagen
    sobel_x = cv2.Sobel(grayscale_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(grayscale_image, cv2.CV_64F, 0, 1, ksize=3)

    # Calcular la magnitud del gradiente
    gradient_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)

    # Calcular la dirección del gradiente
    gradient_direction = np.arctan2(sobel_y, sobel_x)

    # Aplicar la supresión de no máximos
    suppressed_gradient = np.zeros_like(gradient_magnitude)
    for i in range(1, gradient_magnitude.shape[0] - 1):
        for j in range(1, gradient_magnitude.shape[1] - 1):
            angle = gradient_direction[i, j]
            if (angle >= 0 and angle <= np.pi / 4) or (angle >= 7 * np.pi / 4 and angle <= 2 * np.pi):
                if (gradient_magnitude[i, j] >= gradient_magnitude[i, j - 1] and
                    gradient_magnitude[i, j] >= gradient_magnitude[i, j + 1]):
                    suppressed_gradient[i, j] = gradient_magnitude[i, j]
            elif (angle >= np.pi / 4 and angle <= 3 * np.pi / 4):
                if (gradient_magnitude[i, j] >= gradient_magnitude[i - 1, j - 1] and
                    gradient_magnitude[i, j] >= gradient_magnitude[i + 1, j + 1]):
                    suppressed_gradient[i, j] = gradient_magnitude[i, j]
            elif (angle >= 3 * np.pi / 4 and angle <= 5 * np.pi / 4):
                if (gradient_magnitude[i, j] >= gradient_magnitude[i - 1, j] and
                    gradient_magnitude[i, j] >= gradient_magnitude[i + 1, j]):
                    suppressed_gradient[i, j] = gradient_magnitude[i, j]
            else:
                if (gradient_magnitude[i, j] >= gradient_magnitude[i - 1, j + 1] and
                    gradient_magnitude[i, j] >= gradient_magnitude[i + 1, j - 1]):
                    suppressed_gradient[i, j] = gradient_magnitude[i, j]

    # Aplicar la binarización con histéresis (Canny)
    high_threshold = np.max(suppressed_gradient) * high_threshold
    low_threshold = high_threshold * low_threshold

    # Crear una imagen de bordes final
    edge_image = np.zeros_like(suppressed_gradient, dtype=np.uint8)
    strong_edge_i, strong_edge_j = np.where(suppressed_gradient >= high_threshold)
    weak_edge_i, weak_edge_j = np.where((suppressed_gradient >= low_threshold) &
                                        (suppressed_gradient < high_threshold))

    edge_image[strong_edge_i, strong_edge_j] = 255
    edge_image[weak_edge_i, weak_edge_j] = 50  # Valor intermedio para bordes débiles

    return edge_image

Desarrollamos la Transformada de Hough

In [ ]:
# Función para aplicar la Transformada de Hough
def hough_transform(edge_image, theta_resolution=1, rho_resolution=1):
    height, width = edge_image.shape
    max_rho = int(np.sqrt(height**2 + width**2))

    # Rango de valores de theta
    thetas = np.deg2rad(np.arange(-90, 90, theta_resolution))

    # Matriz acumuladora para la Transformada de Hough
    accumulator = np.zeros((2 * max_rho, len(thetas)), dtype=np.uint64)

    # Obtener las coordenadas de los bordes detectados
    edge_coordinates = np.argwhere(edge_image > 0)

    for i in range(len(edge_coordinates)):
        y, x = edge_coordinates[i]
        for t_index in range(len(thetas)):
            rho = int(x * np.cos(thetas[t_index]) + y * np.sin(thetas[t_index]))
            rho_index = rho + max_rho
            accumulator[rho_index, t_index] += 1

    return accumulator, thetas

Cargamos la imagen

In [ ]:
# Cargar la imagen
image = cv2.imread('um_000002.png')

Aplicamos Canny a la imagen

In [ ]:
# Aplicar Canny para detectar bordes
canny_image = canny_edge_detection(image, low_threshold=0.1, high_threshold=0.3)

# Mostrar la imagen con bordes detectados
cv2_imshow(canny_image)

Aplicamos Hough

In [ ]:
# Aplicar la Transformada de Hough para detectar líneas rectas
lines = cv2.HoughLines(canny_image, rho=1, theta=np.pi / 180, threshold=100)

# Dibujar las líneas detectadas en una copia de la imagen original
image_with_lines = np.copy(image)

if lines is not None:
    for line in lines:
        rho, theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * (-b))
        y1 = int(y0 + 1000 * (a))
        x2 = int(x0 - 1000 * (-b))
        y2 = int(y0 - 1000 * (a))
        cv2.line(image_with_lines, (x1, y1), (x2, y2), (0, 0, 255), 2)

# Mostrar la imagen con las líneas detectadas
cv2_imshow(image_with_lines)
